In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Bank Customer Churn Modeling

### First importing the necassery libraries 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns



In [ ]:
#load the data
data = pd.read_csv('../input/churn-for-bank-customers/churn.csv')

print("churn bank modeling data  has {} data points with {} variables each.".format(*data.shape))
data.head()

In [ ]:
#check is there is missing data
data.isnull().sum()

In [ ]:
#check for data formate
data.info()

In [ ]:
#describe the non-numerical data
data[['Geography','Gender']].describe()

# 2.Visualization


#1-Proportion of Customer churn.
#2-Relation between 'Geography' and 'Exited'.
#3-Relation between 'Gender' and 'Exited'.
#4-Relation between 'HasCrCard' and 'Exited'.
#5-Relation between 'IsActiveMember' and 'Exited'.
#6-Relation between 'Age' and 'Exited'

In [ ]:
## visualization of the features 
data.hist(figsize=(10,20), xrot=-45)
plt.show()

In [ ]:
#Create a list that contains the number of exited customer 
ExitedList = [data.Exited[data['Exited']==1].count(), data.Exited[data['Exited']==0].count()]

#set figure size and title
plt.subplots(figsize=(10, 10))
plt.title('Proportion of Customer Churn', size = 12)

#display the proportion of Customer Churn
plt.pie(ExitedList,labels = ['Exited', 'Not Exited'], autopct='%.2f%%', explode = (0 , 0.05))
plt.legend(labels = ['Exited', 'Not Exited'], loc = "upper right")
plt.show()

In [ ]:
#preparing the figure size 
fig, axarr = plt.subplots(2, 3, figsize=(15, 15))

#visulazie the count of Exited and NotExited for each feature

sns.countplot('Geography', hue = 'Exited',data = data, ax = axarr[0][0])
sns.countplot('Gender', hue = 'Exited',data = data, ax = axarr[0][1])
sns.countplot('Tenure', hue = 'Exited',data = data, ax = axarr[0][2])
sns.countplot('NumOfProducts', hue = 'Exited',data = data, ax = axarr[1][0])
sns.countplot('HasCrCard', hue = 'Exited',data = data, ax = axarr[1][1])
sns.countplot('IsActiveMember', hue = 'Exited',data = data, ax = axarr[1][2])


In [ ]:
#visulaization relation between Age and Exited
FacetGrid = sns.FacetGrid(data, hue='Exited', aspect=4)
FacetGrid.map(sns.kdeplot, 'Age', shade=True )
FacetGrid.set(xlim=(16, data['Age'].max()))
FacetGrid.add_legend(labels = ['Stay', "Exited"])

In [ ]:
#Vislization of the count of each value in Geography featrure
sns.countplot(y=data['Geography'], data=data, order = data['Geography'].value_counts().index)
plt.show()
#Vislization of the count of each value in Gender feature

sns.countplot(y=data['Gender'], data=data, order = data['Gender'].value_counts().index)
plt.show()


-check for colleration between data 


In [ ]:
plt.figure(figsize=(7,7))
sns.heatmap(data.corr(), cmap="YlGnBu")

#there is colleration between balance and number of products 
#and also Age is making colleration between it and exited
#and also there is colleration between isActive member and exited
#almost there is no colleration between CustomerId and Exited
#almost there is no colleration between HasCrCard and Exited

### showing the colleration value of each of them 

In [ ]:
data.corr().at['IsActiveMember', 'Exited']


In [ ]:
data.corr().at['Balance', 'NumOfProducts']


In [ ]:
data.corr().at['Age', 'Exited']


# 3-Preprocessing on data

In [ ]:
data.head()

### Clean the data

In [ ]:
#Drop the Surname,CustmerId,HasCrCard features from the data set as it will not considert in prediction 
data.drop("CustomerId", axis=1, inplace=True)
data.drop("Surname", axis=1, inplace=True)
data.drop("HasCrCard", axis=1, inplace=True)

In [ ]:
#apply one hot encodeing to  Gender and Geography
data = pd.get_dummies(data = data ,columns=['Gender', 'Geography'])
data.head()

In [ ]:
#rename the coulmuns
data.rename(columns={'Gender_Female':'Female',
                     'Gender_Male':'Male',
                     'Geography_France': 'France',
                     'Geography_Germany' : 'Germany',
                     'Geography_Spain':'Spain'}, inplace = True)
data.head()

# Split The Data For Training and Testing

In [ ]:
#preparing data for split and prediction
features = data.drop(['Exited'], axis = 1)
predict = data['Exited']

In [ ]:
from sklearn.model_selection import train_test_split 

#split tha data
X_train, X_test, y_train, y_test = train_test_split(features, predict, test_size = 0.4,random_state = 200)
# Show the results of the split
print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))

# Model Evalution

In [ ]:
#improting necassery libraries
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier as DecisionTree
from sklearn.metrics import roc_auc_score,confusion_matrix,f1_score,accuracy_score

In [ ]:
# using Logistic Regresiion algorithm to build first model
logreg_model = LogisticRegression(solver='liblinear')
fit = logreg_model.fit(X_train, y_train)
fit_prediction_train = fit.predict(X_train)
fit_prediction_test = fit.predict(X_test)

#validation of  Logistic Regresiion
print('Accuracy Score for testing set :' +'{}'.format(accuracy_score(fit_prediction_test, y_test)))
print('Accuracy Score for training set :' +'{}'.format(accuracy_score(fit_prediction_train, y_train)))

print('\nF1 score for testing set :'+'{}'.format(f1_score(fit_prediction_test, y_test)))
print('F1 score for training set:'+'{}'.format(f1_score(fit_prediction_train, y_train)))


print('\nROC AUC Score for testing set: ' +'{}'.format(roc_auc_score(y_test, fit.predict_proba(X_test)[::,1])))
print('ROC AUC Score for train set: ' +'{}'.format(roc_auc_score(y_train, fit.predict_proba(X_train)[::,1])))

confusion_matrix(y_test, fit.predict(X_test))

In [ ]:
# using RandomForestClassifier algorithm to build second model
rf = RandomForestClassifier(n_estimators=100)
rf_fit = rf.fit(X_train, y_train)
rf_prediction_train_ = rf_fit.predict(X_train)
rf_prediction_test = rf_fit.predict(X_test)

#validation of RandomForestClassifier
print('Accuracy Score for testing set :' +'{}'.format(accuracy_score(rf_prediction_test, y_test)))
print('Accuracy Score for training set :' +'{}'.format(accuracy_score(rf_prediction_train_, y_train)))

print('\nF1 score for testing set :'+'{}'.format(f1_score(rf_prediction_test, y_test)))
print('F1 score for training set:'+'{}'.format(f1_score(rf_prediction_train_, y_train)))


print('\nROC AUC Score for testing set: ' +'{}'.format(roc_auc_score(y_test, fit.predict_proba(X_test)[::,1])))
print('ROC AUC Score for train set: ' +'{}'.format(roc_auc_score(y_train, fit.predict_proba(X_train)[::,1])))

confusion_matrix(y_test, fit.predict(X_test))

In [ ]:
# using GradientBoostingClassifier algorithm to build second model

gb = GradientBoostingClassifier(n_estimators=100)
gb_fit = gb.fit(X_train, y_train)
gb_prediction_train_ = gb_fit.predict(X_train)
gb_prediction_test = gb_fit.predict(X_test)

#validation of GradientBoostingClassifier
print('Accuracy Score for testing set :' +'{}'.format(accuracy_score(gb_prediction_test, y_test)))
print('Accuracy Score for training set :' +'{}'.format(accuracy_score(gb_prediction_train_, y_train)))

print('\nF1 score for testing set :'+'{}'.format(f1_score(gb_prediction_test, y_test)))
print('F1 score for training set:'+'{}'.format(f1_score(gb_prediction_train_, y_train)))


print('\nROC AUC Score for testing set: ' +'{}'.format(roc_auc_score(y_test, fit.predict_proba(X_test)[::,1])))
print('ROC AUC Score for train set: ' +'{}'.format(roc_auc_score(y_train, fit.predict_proba(X_train)[::,1])))

confusion_matrix(y_test, fit.predict(X_test))


# Resampling the data

In [ ]:
# store No. of Exited and indices
Exited_records = data['Exited'].sum()
Exited_indices = np.array(data[data.Exited == 1].index)

# Picking the indices of the normal Exited
normal_indices = data[data.Exited == 0].index

# Out of the indices we picked, randomly select number of normal records = number of Exited records 
random_normal_indices = np.random.choice(normal_indices, Exited_records, replace = False)
random_normal_indices = np.array(random_normal_indices)

# Merge the 2 indices
under_sample_indices = np.concatenate([Exited_indices,random_normal_indices])

# Copy under sample dataset  
under_sample_data = data.iloc[under_sample_indices,:]

# Split data into features and target labels 
features_undersample = under_sample_data.drop(['Exited'], axis = 1)
target_undersample = under_sample_data['Exited']

# Show ratio
print("Percentage of NotExited: ", under_sample_data.Exited[under_sample_data['Exited'] == 0].count())
print("Percentage of Exited: ", under_sample_data.Exited[under_sample_data['Exited'] == 1].count())
print("Total number of  resampled data: ", under_sample_data['Exited'].count())

In [ ]:
under_sample_Exited_Real = [under_sample_data.Exited[under_sample_data['Exited'] == 0].count(), Exited_records]

# Plot the proportion 
plt.subplots(figsize = (7, 7))
plt.title("Proportion of Exited after resampling data", size = 20)
ax = sns.countplot(x = under_sample_data['Exited'], data= under_sample_data)
ax.legend(labels=['NotExited', 'Exited'], loc = 'upper left')

In [ ]:
# Split the 'features_undersample' and 'target_anderSample' data into training and testing sets
X_train_sampled, X_test_sampled, y_train_sampled, y_test_sampled = train_test_split(features_undersample,
                                                                                    target_undersample,
                                                                                    test_size = 0.15,
                                                                                    random_state = 25)
# Show the results of the split
print("Training set has {} samples.".format(X_train_sampled.shape[0]))
print("Testing set has {} samples.".format(X_test_sampled.shape[0]))

In [ ]:
# using Logistic Regresiion algorithm to fit it on the resampled data
logreg_model = LogisticRegression(solver='liblinear')
fit = logreg_model.fit(X_train_sampled, y_train_sampled)
fit_prediction_train_sampled = fit.predict(X_train_sampled)
fit_prediction_test_sampled = fit.predict(X_test_sampled)

#validation of  Logistic Regresiion
print('Accuracy Score for testing set :' +'{}'.format(accuracy_score(fit_prediction_test_sampled, y_test_sampled)))
print('Accuracy Score for training set :' +'{}'.format(accuracy_score(fit_prediction_train_sampled, y_train_sampled)))

print('\nF1 score for testing set :'+'{}'.format(f1_score(fit_prediction_test_sampled, y_test_sampled)))
print('F1 score for training set:'+'{}'.format(f1_score(fit_prediction_train_sampled, y_train_sampled)))


print('\nROC AUC Score for testing set: ' +'{}'.format(roc_auc_score(y_test_sampled, fit.predict_proba(X_test_sampled)[::,1])))
print('ROC AUC Score for train set: ' +'{}'.format(roc_auc_score(y_train_sampled, fit.predict_proba(X_train_sampled)[::,1])))

confusion_matrix(y_test_sampled, fit.predict(X_test_sampled))

In [ ]:
# using RandomForestClassifier algorithm to to fit it on the resampled data
rf = RandomForestClassifier(n_estimators=100)
rf_fit = rf.fit(X_train_sampled, y_train_sampled)
rf_prediction_train_sampled = rf_fit.predict(X_train_sampled)
rf_prediction_test_sampled= rf_fit.predict(X_test_sampled)

#validation of RandomForestClassifier
print('Accuracy Score for testing set :' +'{}'.format(accuracy_score(rf_prediction_test_sampled, y_test_sampled)))
print('Accuracy Score for training set :' +'{}'.format(accuracy_score(rf_prediction_train_sampled, y_train_sampled)))

print('\nF1 score for testing set :'+'{}'.format(f1_score(rf_prediction_test_sampled, y_test_sampled)))
print('F1 score for training set:'+'{}'.format(f1_score(rf_prediction_train_sampled, y_train_sampled)))


print('\nROC AUC Score for testing set: ' +'{}'.format(roc_auc_score(y_test_sampled, fit.predict_proba(X_test_sampled)[::,1])))
print('ROC AUC Score for train set: ' +'{}'.format(roc_auc_score(y_train_sampled, fit.predict_proba(X_train_sampled)[::,1])))

confusion_matrix(y_test_sampled, fit.predict(X_test_sampled))


In [ ]:
gb = GradientBoostingClassifier(n_estimators=100)
gb_fit = gb.fit(X_train, y_train)
gb_prediction_train_sampled = gb_fit.predict(X_train_sampled)
gb_prediction_test_sampled = gb_fit.predict(X_test_sampled)

#validation of GradientBoostingClassifier
print('Accuracy Score for testing set :' +'{}'.format(accuracy_score(gb_prediction_test_sampled, y_test_sampled)))
print('Accuracy Score for training set :' +'{}'.format(accuracy_score(gb_prediction_train_sampled, y_train_sampled)))

print('\nF1 score for testing set :'+'{}'.format(f1_score(gb_prediction_test_sampled, y_test_sampled)))
print('F1 score for training set:'+'{}'.format(f1_score(gb_prediction_train_sampled, y_train_sampled)))


print('\nROC AUC Score for testing set: ' +'{}'.format(roc_auc_score(y_test_sampled, fit.predict_proba(X_test_sampled)[::,1])))
print('ROC AUC Score for train set: ' +'{}'.format(roc_auc_score(y_train_sampled, fit.predict_proba(X_train_sampled)[::,1])))

confusion_matrix(y_test_sampled, fit.predict(X_test_sampled))

In [ ]:
# Use the logistic Regresiion model to predict the training and testing set of whole datasetafter 
fit_prediction_train_after_sampled = fit.predict(X_train)
fit_prediction_test_after_sampled = fit.predict(X_test)

print('Accuracy Score for testing set :' +'{}'.format(accuracy_score(fit_prediction_test_after_sampled, y_test)))
print('Accuracy Score for training set :' +'{}'.format(accuracy_score(fit_prediction_train_after_sampled, y_train)))

print('\nF1 score for testing set :'+'{}'.format(f1_score(fit_prediction_test_after_sampled, y_test)))
print('F1 score for training set:'+'{}'.format(f1_score(fit_prediction_train_after_sampled, y_train)))


print('\nROC AUC Score for testing set: ' +'{}'.format(roc_auc_score(y_test, fit_prediction_test_after_sampled)))
print('ROC AUC Score for train set: ' +'{}'.format(roc_auc_score(y_train,fit_prediction_train_after_sampled )))


In [ ]:
# Use the Random ForestClassifier model to predict the training and testing set of whole dataset 
rf_fit_prediction_train_after_sampled = rf_fit.predict(X_train)
rf_fit_prediction_test_after_sampled = rf_fit.predict(X_test)

print('Accuracy Score for testing set :' +'{}'.format(accuracy_score(rf_fit_prediction_test_after_sampled, y_test)))
print('Accuracy Score for training set :' +'{}'.format(accuracy_score(rf_fit_prediction_train_after_sampled, y_train)))

print('\nF1 score for testing set :'+'{}'.format(f1_score(rf_fit_prediction_test_after_sampled, y_test)))
print('F1 score for training set:'+'{}'.format(f1_score(rf_fit_prediction_train_after_sampled, y_train)))


print('\nROC AUC Score for testing set: ' +'{}'.format(roc_auc_score(y_test, rf_fit_prediction_test_after_sampled)))
print('ROC AUC Score for train set: ' +'{}'.format(roc_auc_score(y_train,rf_fit_prediction_train_after_sampled )))



In [ ]:
# Use this GradientBoostingClassifier model to predict the training and testing set of whole dataset 

gb_fit_prediction_train_after_sampled = gb_fit.predict(X_train)
gb_fit_prediction_test_after_sampled = gb_fit.predict(X_test)

print('Accuracy Score for testing set :' +'{}'.format(accuracy_score(gb_fit_prediction_test_after_sampled, y_test)))
print('Accuracy Score for training set :' +'{}'.format(accuracy_score(gb_fit_prediction_train_after_sampled, y_train)))

print('\nF1 score for testing set :'+'{}'.format(f1_score(gb_fit_prediction_test_after_sampled, y_test)))
print('F1 score for training set:'+'{}'.format(f1_score(gb_fit_prediction_train_after_sampled, y_train)))


print('\nROC AUC Score for testing set: ' +'{}'.format(roc_auc_score(y_test, gb_fit_prediction_test_after_sampled)))
print('ROC AUC Score for train set: ' +'{}'.format(roc_auc_score(y_train,gb_fit_prediction_train_after_sampled )))
